# Premières requêtes sur API SIRENE

## Utilisation de la librairie requests
[La documentation officielle](http://docs.python-requests.org/en/v0.10.6/api/) en anglais, [une version en français](http://fr.python-requests.org/en/latest/user/quickstart.html) et un [tutoriel](http://dridk.me/python-requests.html) également en français. 

In [1]:
import requests
import pandas

## Une première requête

Construire un requête est simple :
* on définit l'URL de la requête (sous forme de chaîne de caractère),

In [2]:
url = "https://prototype.api-sirene.insee.fr/ws/siret/"

* si besoin est, on définit les paramètres de la requête (sous forme de dictionnaire),

In [3]:
##On a choisit pour l'exemple de s'en prendre aux notaires (oui c'est gratuit, non je n'ai pas envie d'en parler...).
q_param_value = 'Denomination:Notaire'
requete_params = {'q':q_param_value}

* si besoin est, on définit les urls des proxies (sous forme de dictionnaire)

In [4]:
insee_proxies = {'http': 'http://proxy-rie.http.insee.fr:8080',
              'https': 'http://proxy-rie.http.insee.fr:8080'}

* on passe le tout en paramètre de la fonction correspondant au verbe HTTP souhaité (get, post, put, delete... et selon le contexte avec ou sans le param proxies)

In [6]:
requete = requests.get(url, params=requete_params)

On peut vérifier que la requête s'est bien passée soit :
* en vérifiant la valeur du code retour qui doit valoir 200

In [7]:
assert requete.status_code == 200, "Erreur lors de la requête"

* ou en utilisant une fonction ad-hoc

In [8]:
requete.raise_for_status()
##Aucun retour si la requête a renvoyé 200

## Accéder au contenu de la requête

API SIRENE renvoie du JSON (oui, il y a tout le JSON, mais ainsi vous l'aurez déjà vu une fois).

In [9]:
requete.text

'{"Header":{"Statut":200,"Message":"ok","Total":831,"Debut":0,"Nombre":20},"Etablissements":[{"Siren":"792798415","Nic":"00016","StatutDiffusionEtablissement":"O","DateCreationEtablissement":"2013-04-01","NombrePeriodes":1,"UniteLegale":{"StatutDiffusionUniteLegale":"O","DateCreationUniteLegale":"2013-04-01","Sigle":null,"Sexe":null,"NomUsage":null,"Prenom1":null,"Prenom2":null,"Prenom3":null,"Prenom4":null,"PrenomUsuel":null,"CaractereEmployeur":"N","EtatAdministratif":"A","Nom":null,"Denomination":"NOTAIRE","ActivitePrincipale":"01.11Z","LibelleActivite":"Culture de céréales (à l\'exception du riz), de légumineuses et de graines oléagineuses","NomenclatureActivite":"NAFRev2","CategorieJuridique":"6597","CategorieProfessionnelle":null,"NICSiege":"00016"},"Adresse":{"ComplementAdresseEtablissement":null,"NumeroVoieEtablissement":"2","IndiceRepetitionEtablissement":null,"TypeVoieEtablissement":"CHE","LibelleVoieEtablissement":"DE L ORME","CodePostalEtablissement":"51350","LibelleCommune

Pour pouvoir manipuler les données, il faut d'abord les 'parser'...   
Pour Python il s'agit encore du résultat d'une requête HTTP.

In [10]:
requete.keys()

AttributeError: 'Response' object has no attribute 'keys'

Parser le contenu de la réponse est triviale (quand le contenu/body respecte un format supporté)

In [11]:
data_raw = requete.json()

In [12]:
print(data_raw)

{'Header': {'Statut': 200, 'Message': 'ok', 'Total': 831, 'Debut': 0, 'Nombre': 20}, 'Etablissements': [{'Siren': '792798415', 'Nic': '00016', 'StatutDiffusionEtablissement': 'O', 'DateCreationEtablissement': '2013-04-01', 'NombrePeriodes': 1, 'UniteLegale': {'StatutDiffusionUniteLegale': 'O', 'DateCreationUniteLegale': '2013-04-01', 'Sigle': None, 'Sexe': None, 'NomUsage': None, 'Prenom1': None, 'Prenom2': None, 'Prenom3': None, 'Prenom4': None, 'PrenomUsuel': None, 'CaractereEmployeur': 'N', 'EtatAdministratif': 'A', 'Nom': None, 'Denomination': 'NOTAIRE', 'ActivitePrincipale': '01.11Z', 'LibelleActivite': "Culture de céréales (à l'exception du riz), de légumineuses et de graines oléagineuses", 'NomenclatureActivite': 'NAFRev2', 'CategorieJuridique': '6597', 'CategorieProfessionnelle': None, 'NICSiege': '00016'}, 'Adresse': {'ComplementAdresseEtablissement': None, 'NumeroVoieEtablissement': '2', 'IndiceRepetitionEtablissement': None, 'TypeVoieEtablissement': 'CHE', 'LibelleVoieEtabli

Maintenant on dispose d'un dictionnaire Python classique (cf. [les dictionnaires Python](https://www.tutorialspoint.com/python/python_dictionary.htm) au besoin)

In [13]:
data_raw.keys()

dict_keys(['Header', 'Etablissements'])

Le contenu qui nous intéresse correspond à la clé Etablissements, qui elle contient une liste des établissements correspondant à la requête.

In [14]:
data = data_raw["Etablissements"]
type(data)

list

## Autres requêtes

On peut aussi requêter l'API pour avoir l'historique complet (entreprise existante et fermée) à partir de la racine 'siren'.

In [18]:
url2 = "https://prototype.api-sirene.insee.fr/ws/siren/"
bad_requete2 = requests.get(url2 , params=requete_params)
bad_requete2.raise_for_status()

HTTPError: 400 Client Error: Bad Request for url: https://prototype.api-sirene.insee.fr/ws/siren/?q=Denomination%3ANotaire

``Attention`` les paramètres attendus non pas tout à fait la même syntaxe...

In [19]:
q2_param_value = 'periode(Denomination:Notaire)'
requete2_params = {'q':q2_param_value}
requete2 = requests.get(url2 , params=requete2_params)
requete2.raise_for_status()

Cette fois une clé Periodes renseigne sur les différents états des unités légales renvoyées.